<a href="https://colab.research.google.com/github/jmbaek/NLP_applications/blob/main/PretrainingBERT_PythonCodeTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 9.7 MB/s 
     |████████████████████████████████| 462 kB 64.9 MB/s 


In [3]:
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [7]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [15]:
text_test = ['that was a nice movie indeed', 'I love python programming']
text_processed = bert_preprocess_model(text_test)
text_processed.keys()

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])

In [18]:
text_processed['input_word_ids']

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[  101,  2008,  2001,  1037,  3835,  3185,  5262,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [19]:
bert_model = hub.KerasLayer(encoder_url)
bert_results = bert_model(text_processed)
bert_results.keys()

dict_keys(['pooled_output', 'default', 'sequence_output', 'encoder_outputs'])

In [24]:
bert_results['sequence_output']

<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[ 0.08247545, -0.02013815,  0.14680883, ..., -0.05504194,
          0.21207508,  0.11869051],
        [-0.46663198, -0.24285477, -0.3423316 , ..., -0.2831246 ,
          1.3741698 ,  0.19046658],
        [-0.27740252, -0.57036436,  0.24329393, ...,  0.23598474,
          0.2682978 ,  0.26211837],
        ...,
        [ 0.17072722,  0.01922826,  0.5363991 , ...,  0.24514116,
          0.2635091 , -0.04776914],
        [ 0.21648622,  0.04456874,  0.547078  , ...,  0.2789396 ,
          0.2860774 , -0.07193862],
        [ 0.08261422,  0.07403944,  0.5833179 , ...,  0.26967448,
          0.19498406, -0.02856611]],

       [[-0.07900561,  0.36335143, -0.21101603, ..., -0.17183733,
          0.16299734,  0.6724263 ],
        [ 0.27883607,  0.43716288, -0.35764828, ..., -0.0446361 ,
          0.38315198,  0.58879817],
        [ 1.2037672 ,  1.0727026 ,  0.4840874 , ...,  0.24921   ,
          0.40730882,  0.4048173 ],
        ...,

In [23]:
bert_results['encoder_outputs'][-1] == bert_results['sequence_output']

<tf.Tensor: shape=(2, 128, 768), dtype=bool, numpy=
array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])>

## Spam detection

In [39]:
!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Copy the API command from kaggle
!kaggle datasets download -d venky73/spam-mails-dataset
!unzip -q /content/spam-mails-dataset.zip -d data

spam-mails-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
!pip install tensorflow_text

In [37]:
import tensorflow as tf
import tensorflow_hub as hup
import tensorflow_text as text
import pandas as pd

In [41]:
df = pd.read_csv("data/spam.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [42]:
df.groupby('label').describe()

Unnamed: 0                                                         \
           count    mean          std     min      25%     50%      75%   
label                                                                     
ham       3672.0  1835.5  1060.159422     0.0   917.75  1835.5  2753.25   
spam      1499.0  4421.0   432.868340  3672.0  4046.50  4421.0  4795.50   

              label_num                                     
          max     count mean  std  min  25%  50%  75%  max  
label                                                       
ham    3671.0    3672.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
spam   5170.0    1499.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [43]:
df['label'].value_counts()

ham     3672
spam    1499
Name: label, dtype: int64

In [46]:
# Imbalance data treatment
df_spam = df[df['label'] == 'spam']
df_spam.shape


(1499, 4)

In [45]:
df_ham = df[df['label'] == 'ham']
df_ham.shape

(3672, 4)

In [47]:
df_ham.sample(df_spam.shape[0])

,Unnamed: 0,label,text,label_num
1914,1267,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
4253,1169,ham,Subject: leak on wilson creek lateral\r\nrepai...,0
1246,518,ham,Subject: williams energy marketing\r\ni have a...,0
1673,2244,ham,Subject: deal extension for 11 / 21 / 2000 for...,0
2578,3565,ham,"Subject: need deal for march 2000\r\ndaren ,\r...",0
...,...,...,...,...
2158,4,ham,Subject: meter 7268 nov allocation\r\nfyi .\r\...,0
3554,3149,ham,Subject: natural gas nomination for 5 / 01\r\n...,0
2621,2834,ham,Subject: neon lesson 5\r\nhere is lesson # 5 ....,0
1213,2485,ham,Subject: re : coastal ctr # 96008903 meter 098...,0
